https://www.kaggle.com/competitions/games-rating/data
Предсказать средний рейтинг (Rating Average) настольных игр на основе их характеристик.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
import warnings
warnings.filterwarnings('ignore')

train_data = pd.read_csv('C:/study-1/Proga-5/3_ml/train_data.csv')
test_data = pd.read_csv('C:/study-1/Proga-5/3_ml/test_data.csv')

# Преобразование типов данных
def convert_dtypes(df):
    df_conv = df.copy()
    
    # Преобразуем числовые колонки
    numeric_columns = ['Rating Average', 'Complexity Average']
    for col in numeric_columns:
        if col in df_conv.columns:
            # Заменяем запятые на точки и преобразуем в float
            df_conv[col] = df_conv[col].astype(str).str.replace(',', '.').astype(float)
    
    # Преобразуем другие числовые колонки
    if 'ID' in df_conv.columns:
        df_conv['ID'] = pd.to_numeric(df_conv['ID'], errors='coerce')
    if 'Year Published' in df_conv.columns:
        df_conv['Year Published'] = pd.to_numeric(df_conv['Year Published'], errors='coerce')
    if 'Owned Users' in df_conv.columns:
        df_conv['Owned Users'] = pd.to_numeric(df_conv['Owned Users'], errors='coerce')
    
    return df_conv

# Преобразуем типы данных
train_data = convert_dtypes(train_data)
test_data = convert_dtypes(test_data)

print(train_data.isnull().sum())

# Функция для предобработки данных
def preprocess_data(df, is_train=True):
    df_processed = df.copy()
    
    # Обработка категориальных признаков
    if 'Mechanics' in df_processed.columns:
        df_processed['Mechanics_Count'] = df_processed['Mechanics'].apply(
            lambda x: len(str(x).split(',')) if pd.notnull(x) and str(x) != 'nan' else 0
        )
    
    if 'Domains' in df_processed.columns:
        df_processed['Domains_Count'] = df_processed['Domains'].apply(
            lambda x: len(str(x).split(',')) if pd.notnull(x) and str(x) != 'nan' else 0
        )
    
    # Создание новых признаков
    if 'Min Players' in df_processed.columns and 'Max Players' in df_processed.columns:
        df_processed['Player_Range'] = df_processed['Max Players'] - df_processed['Min Players']
        df_processed['Players_Avg'] = (df_processed['Min Players'] + df_processed['Max Players']) / 2
    
    # Возраст игры
    current_year = 2021
    if 'Year Published' in df_processed.columns:
        df_processed['Game_Age'] = current_year - df_processed['Year Published']
        df_processed['Game_Age'] = df_processed['Game_Age'].clip(lower=0)
        df_processed['Is_Old_Game'] = (df_processed['Game_Age'] > 20).astype(int)
    
    # сделать данные более нормальными- логорифмирование
    if 'Users Rated' in df_processed.columns:
        df_processed['Log_Users_Rated'] = np.log1p(df_processed['Users Rated'])
    
    if 'Owned Users' in df_processed.columns:
        df_processed['Log_Owned_Users'] = np.log1p(df_processed['Owned Users'])
    
    if 'BGG Rank' in df_processed.columns:
        # Обработка рейтинга (меньше число = лучше игра)
        df_processed['BGG_Rank_Log'] = np.log1p(df_processed['BGG Rank'])
        df_processed['Is_Top_100'] = (df_processed['BGG Rank'] <= 100).astype(int)
        df_processed['Is_Top_500'] = (df_processed['BGG Rank'] <= 500).astype(int)
    
    # Отношение сложности к минимальному возрасту.
    if 'Complexity Average' in df_processed.columns and 'Min Age' in df_processed.columns:
        df_processed['Complexity_Age_Ratio'] = df_processed['Complexity Average'] / df_processed['Min Age'].replace(0, 1)  # Защита от деления на 0
    
    if 'Play Time' in df_processed.columns:
        # Категории по времени игры
        df_processed['Play_Time_Category'] = pd.cut(
            df_processed['Play Time'],
            bins=[0, 30, 60, 120, 300, float('inf')],
            labels=['Very_Short', 'Short', 'Medium', 'Long', 'Very_Long']
        )
        play_time_dummies = pd.get_dummies(df_processed['Play_Time_Category'], prefix='Play_Time')
        df_processed = pd.concat([df_processed, play_time_dummies], axis=1)
    
    # Удаление исходных категориальных колонок которые были преобразованы
    columns_to_drop = ['Mechanics', 'Domains', 'Play_Time_Category', 'Name']
    for col in columns_to_drop:
        if col in df_processed.columns:
            df_processed.drop(col, axis=1, inplace=True)
    
    return df_processed

# Предобработка данных
train_processed = preprocess_data(train_data, is_train=True)
test_processed = preprocess_data(test_data, is_train=False)

all_features = [col for col in train_processed.columns if col not in ['Rating Average', 'ID']]

# Подготовка данных для обучения
X = train_processed[all_features]
y = train_processed['Rating Average']

# Заполнение пропущенных значений
imputer = SimpleImputer(strategy='median')
X_imputed = imputer.fit_transform(X)
test_imputed = imputer.transform(test_processed[all_features])

# Масштабирование признаков - Приводим все признаки к одинаковому масштабу
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_imputed)
test_scaled = scaler.transform(test_imputed)

# Разделение на тренировочную и валидационную выборку
X_train, X_val, y_train, y_val = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, shuffle=True
)

# Обучение нескольких моделей
models = {
    'RandomForest': RandomForestRegressor(
        n_estimators=200,
        max_depth=20,
        min_samples_split=5,
        min_samples_leaf=2,
        random_state=42,
        n_jobs=-1
    ),
    'GradientBoosting': GradientBoostingRegressor(
        n_estimators=200,
        max_depth=6,
        learning_rate=0.1,
        random_state=42
    )
}

# Обучение и оценка моделей
best_model = None
best_mse = float('inf')
best_model_name = ""

print("\nОбучение моделей...")
for name, model in models.items():
    print(f"Обучение {name}...")
    model.fit(X_train, y_train)
    
    # Предсказание на валидационной выборке
    y_pred_val = model.predict(X_val)
    
    # Расчет MSE
    mse = mean_squared_error(y_val, y_pred_val)
    print(f"{name} - MSE: {mse:.4f}")
    
    if mse < best_mse:
        best_mse = mse
        best_model = model
        best_model_name = name


# Кросс-валидация для лучшей модели
from sklearn.model_selection import cross_val_score

print("\nПроводим кросс-валидацию...")
cv_scores = cross_val_score(best_model, X_scaled, y, cv=5, scoring='neg_mean_squared_error')
cv_mse = -cv_scores.mean()

# Обучение лучшей модели на всех данных
best_model.fit(X_scaled, y)

# Предсказание на тестовых данных
test_predictions = best_model.predict(test_scaled)

# Постобработка предсказаний
# Ограничиваем предсказания разумными пределами для рейтингов игр
test_predictions = np.clip(test_predictions, 1.0, 10.0)

# Создание файла
submission = pd.DataFrame({
    'index': test_data['ID'] if 'ID' in test_data.columns else range(len(test_predictions)),
    'Rating Average': test_predictions
})

# Сохранение файла
submission_file = 'C:/study-1/Proga-5/3_ml/submission.csv'
submission.to_csv(submission_file, index=False)


# Анализ важности признаков
if hasattr(best_model, 'feature_importances_'):
    feature_importance = pd.DataFrame({
        'feature': all_features,
        'importance': best_model.feature_importances_
    }).sort_values('importance', ascending=False)

# Анализ предсказаний на тренировочных данных
train_predictions = best_model.predict(X_scaled)
train_mse = mean_squared_error(y, train_predictions)
